In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange

In [2]:
import torch
import torch.nn as nn

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

data_type=torch.float32

Using cuda device


In [4]:
x0 = 0# initial condition
sigma = 1 # volatility
segs=5
d=20
r =0 # risk free rate
batch_size = 100 # batch size
steps=100
T = 1 # maturity
dt = T/steps # mesh size
dt = T/steps # mesh size
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level

In [5]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [56]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,d))*x0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec

def ComputeMultiLevelSig(path, number_of_segment, depth,log_sig=False):
    n_batch, nsteps, chanels = path.shape
    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []
    s=iisig.prepare(d+1,depth)
    
    if log_sig: 
        ll=iisig.logsig(np.expand_dims(path[:,0,:],axis=1),s)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.logsig(path[:,0:t_vec[i+1]+1,:],s)) ##if not
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    else: 
        ll=iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.sig(path[:,0:t_vec[i+1]+1,:],depth)) ##if not
            #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
        
    return MultiLevelSig

In [57]:
dW=np.sqrt(dt)*np.random.normal(size=(100, steps,d))
bm_p,s_path=create_stock(x0,r,sigma,T,steps,100,dW)
tv=jointime(T,bm_p)
ms=ComputeMultiLevelSig(tv, segs, 3,log_sig=True)

In [58]:
dW.shape,bm_p.shape,ms.shape

((100, 100, 20), (100, 101, 20), (100, 6, 3311))

In [59]:
def generate_samples(batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    sigs=ComputeMultiLevelSig(S_timePath, segs, 3,True)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=terminal_f(x_ten)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [60]:
def terminal_f(x_ten):
    res=(torch.sum(x_ten[:,:-1,:],dim=(1,2))*dt)**2
    return res

In [61]:
mm=generate_samples(batch_in=100)
sigs, dW, yy,x_ten, sel=mm

In [62]:
class Config(object):
    n_layer = 4
    
    dim=3311; 
    Ntime=segs; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,1] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

### Determine the terminal loss

In [78]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.relu(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
    #    self.Y0=Parameter(torch.tensor([0.0])) #torch.rand(500,1)
        self.Y0=Parameter(torch.ones(100,1)*6.0) ## Has to do with the batch_size
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        
    def forward(self,batch_sig,batch_dW): 
   #     Z_path=self.model(batch_sig)
        Y=self.Y0
        for i in range(segs):
            Y=Y+sigma*torch.sum(self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
        return Y

## Check the definution of the final loss function

In [79]:
def Loss(Y_pred,Y_true):
    res=torch.mean((Y_pred-Y_true)**2)
    return res

In [80]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE=Lookback_PPDE(cfg)
optimizer=optim.Adam(model_PPDE.parameters(),lr=5e-4)

In [81]:
y0_mean=[];
loss_vec=[];
grad_clip=0.2

In [77]:
for i in range(2000):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=100)
    

    x_temp=model_PPDE(batch_sig,batch_dw)
    loss_temp=Loss(x_temp, batch_y)
    
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE.parameters(), grad_clip)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
    
    y0_val=model_PPDE.Y0.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)

    if i%10==0:
      #  print("Iter:", i,  loss_temp) 
        print("Iter:", i,  model_PPDE.Y0.mean(), loss_temp)

Iter: 0 tensor(6.7005, grad_fn=<MeanBackward0>) tensor(261.2284, grad_fn=<MeanBackward0>)
Iter: 10 tensor(6.7007, grad_fn=<MeanBackward0>) tensor(150.3068, grad_fn=<MeanBackward0>)
Iter: 20 tensor(6.6997, grad_fn=<MeanBackward0>) tensor(78.4753, grad_fn=<MeanBackward0>)
Iter: 30 tensor(6.6983, grad_fn=<MeanBackward0>) tensor(88.2935, grad_fn=<MeanBackward0>)
Iter: 40 tensor(6.6972, grad_fn=<MeanBackward0>) tensor(114.5349, grad_fn=<MeanBackward0>)
Iter: 50 tensor(6.6973, grad_fn=<MeanBackward0>) tensor(92.5194, grad_fn=<MeanBackward0>)
Iter: 60 tensor(6.6965, grad_fn=<MeanBackward0>) tensor(86.5867, grad_fn=<MeanBackward0>)
Iter: 70 tensor(6.6949, grad_fn=<MeanBackward0>) tensor(131.3831, grad_fn=<MeanBackward0>)
Iter: 80 tensor(6.6942, grad_fn=<MeanBackward0>) tensor(166.4622, grad_fn=<MeanBackward0>)
Iter: 90 tensor(6.6936, grad_fn=<MeanBackward0>) tensor(67.9382, grad_fn=<MeanBackward0>)
Iter: 100 tensor(6.6940, grad_fn=<MeanBackward0>) tensor(44.5635, grad_fn=<MeanBackward0>)
Iter:

Iter: 900 tensor(6.6461, grad_fn=<MeanBackward0>) tensor(82.5366, grad_fn=<MeanBackward0>)
Iter: 910 tensor(6.6448, grad_fn=<MeanBackward0>) tensor(57.5955, grad_fn=<MeanBackward0>)
Iter: 920 tensor(6.6445, grad_fn=<MeanBackward0>) tensor(88.6637, grad_fn=<MeanBackward0>)
Iter: 930 tensor(6.6438, grad_fn=<MeanBackward0>) tensor(94.1518, grad_fn=<MeanBackward0>)
Iter: 940 tensor(6.6439, grad_fn=<MeanBackward0>) tensor(72.6126, grad_fn=<MeanBackward0>)
Iter: 950 tensor(6.6447, grad_fn=<MeanBackward0>) tensor(53.1248, grad_fn=<MeanBackward0>)
Iter: 960 tensor(6.6449, grad_fn=<MeanBackward0>) tensor(89.4846, grad_fn=<MeanBackward0>)
Iter: 970 tensor(6.6442, grad_fn=<MeanBackward0>) tensor(82.7061, grad_fn=<MeanBackward0>)
Iter: 980 tensor(6.6424, grad_fn=<MeanBackward0>) tensor(157.5437, grad_fn=<MeanBackward0>)
Iter: 990 tensor(6.6408, grad_fn=<MeanBackward0>) tensor(134.7805, grad_fn=<MeanBackward0>)
Iter: 1000 tensor(6.6383, grad_fn=<MeanBackward0>) tensor(130.2391, grad_fn=<MeanBackwar

Iter: 1790 tensor(6.6074, grad_fn=<MeanBackward0>) tensor(87.5427, grad_fn=<MeanBackward0>)
Iter: 1800 tensor(6.6078, grad_fn=<MeanBackward0>) tensor(81.5271, grad_fn=<MeanBackward0>)
Iter: 1810 tensor(6.6075, grad_fn=<MeanBackward0>) tensor(49.4529, grad_fn=<MeanBackward0>)
Iter: 1820 tensor(6.6071, grad_fn=<MeanBackward0>) tensor(56.7892, grad_fn=<MeanBackward0>)
Iter: 1830 tensor(6.6060, grad_fn=<MeanBackward0>) tensor(71.1672, grad_fn=<MeanBackward0>)
Iter: 1840 tensor(6.6057, grad_fn=<MeanBackward0>) tensor(70.3998, grad_fn=<MeanBackward0>)
Iter: 1850 tensor(6.6053, grad_fn=<MeanBackward0>) tensor(96.6410, grad_fn=<MeanBackward0>)
Iter: 1860 tensor(6.6036, grad_fn=<MeanBackward0>) tensor(77.3911, grad_fn=<MeanBackward0>)
Iter: 1870 tensor(6.6005, grad_fn=<MeanBackward0>) tensor(67.8545, grad_fn=<MeanBackward0>)
Iter: 1880 tensor(6.5967, grad_fn=<MeanBackward0>) tensor(46.3233, grad_fn=<MeanBackward0>)
Iter: 1890 tensor(6.5944, grad_fn=<MeanBackward0>) tensor(47.6885, grad_fn=<Mean

0.2

In [82]:
for i in range(8000):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=100)
    

    x_temp=model_PPDE(batch_sig,batch_dw)
    loss_temp=Loss(x_temp, batch_y)
    
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE.parameters(), grad_clip)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
    
    y0_val=model_PPDE.Y0.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)

    if i%10==0:
      #  print("Iter:", i,  loss_temp) 
        print("Iter:", i,  model_PPDE.Y0.mean(), loss_temp)

Iter: 0 tensor(5.9999, grad_fn=<MeanBackward0>) tensor(50.3586, grad_fn=<MeanBackward0>)
Iter: 10 tensor(5.9999, grad_fn=<MeanBackward0>) tensor(44.1337, grad_fn=<MeanBackward0>)
Iter: 20 tensor(6.0016, grad_fn=<MeanBackward0>) tensor(62.5448, grad_fn=<MeanBackward0>)
Iter: 30 tensor(6.0035, grad_fn=<MeanBackward0>) tensor(35.7849, grad_fn=<MeanBackward0>)
Iter: 40 tensor(6.0060, grad_fn=<MeanBackward0>) tensor(71.1814, grad_fn=<MeanBackward0>)
Iter: 50 tensor(6.0079, grad_fn=<MeanBackward0>) tensor(54.3673, grad_fn=<MeanBackward0>)
Iter: 60 tensor(6.0104, grad_fn=<MeanBackward0>) tensor(127.8956, grad_fn=<MeanBackward0>)
Iter: 70 tensor(6.0136, grad_fn=<MeanBackward0>) tensor(75.9890, grad_fn=<MeanBackward0>)
Iter: 80 tensor(6.0174, grad_fn=<MeanBackward0>) tensor(62.2536, grad_fn=<MeanBackward0>)
Iter: 90 tensor(6.0199, grad_fn=<MeanBackward0>) tensor(53.2815, grad_fn=<MeanBackward0>)
Iter: 100 tensor(6.0217, grad_fn=<MeanBackward0>) tensor(74.9594, grad_fn=<MeanBackward0>)
Iter: 110

Iter: 900 tensor(6.1918, grad_fn=<MeanBackward0>) tensor(80.1306, grad_fn=<MeanBackward0>)
Iter: 910 tensor(6.1947, grad_fn=<MeanBackward0>) tensor(117.4727, grad_fn=<MeanBackward0>)
Iter: 920 tensor(6.1989, grad_fn=<MeanBackward0>) tensor(92.6967, grad_fn=<MeanBackward0>)
Iter: 930 tensor(6.2025, grad_fn=<MeanBackward0>) tensor(91.1412, grad_fn=<MeanBackward0>)
Iter: 940 tensor(6.2056, grad_fn=<MeanBackward0>) tensor(55.1769, grad_fn=<MeanBackward0>)
Iter: 950 tensor(6.2067, grad_fn=<MeanBackward0>) tensor(36.2559, grad_fn=<MeanBackward0>)
Iter: 960 tensor(6.2062, grad_fn=<MeanBackward0>) tensor(82.1259, grad_fn=<MeanBackward0>)
Iter: 970 tensor(6.2068, grad_fn=<MeanBackward0>) tensor(93.7348, grad_fn=<MeanBackward0>)
Iter: 980 tensor(6.2093, grad_fn=<MeanBackward0>) tensor(56.0763, grad_fn=<MeanBackward0>)
Iter: 990 tensor(6.2122, grad_fn=<MeanBackward0>) tensor(110.7576, grad_fn=<MeanBackward0>)
Iter: 1000 tensor(6.2136, grad_fn=<MeanBackward0>) tensor(63.7613, grad_fn=<MeanBackward

Iter: 1790 tensor(6.3315, grad_fn=<MeanBackward0>) tensor(81.1890, grad_fn=<MeanBackward0>)
Iter: 1800 tensor(6.3325, grad_fn=<MeanBackward0>) tensor(85.8291, grad_fn=<MeanBackward0>)
Iter: 1810 tensor(6.3339, grad_fn=<MeanBackward0>) tensor(74.5303, grad_fn=<MeanBackward0>)
Iter: 1820 tensor(6.3364, grad_fn=<MeanBackward0>) tensor(76.6077, grad_fn=<MeanBackward0>)
Iter: 1830 tensor(6.3370, grad_fn=<MeanBackward0>) tensor(80.4178, grad_fn=<MeanBackward0>)
Iter: 1840 tensor(6.3378, grad_fn=<MeanBackward0>) tensor(71.8914, grad_fn=<MeanBackward0>)
Iter: 1850 tensor(6.3388, grad_fn=<MeanBackward0>) tensor(111.1326, grad_fn=<MeanBackward0>)
Iter: 1860 tensor(6.3400, grad_fn=<MeanBackward0>) tensor(59.2224, grad_fn=<MeanBackward0>)
Iter: 1870 tensor(6.3412, grad_fn=<MeanBackward0>) tensor(61.1533, grad_fn=<MeanBackward0>)
Iter: 1880 tensor(6.3433, grad_fn=<MeanBackward0>) tensor(117.5109, grad_fn=<MeanBackward0>)
Iter: 1890 tensor(6.3451, grad_fn=<MeanBackward0>) tensor(164.6528, grad_fn=<M

Iter: 2680 tensor(6.4190, grad_fn=<MeanBackward0>) tensor(112.8072, grad_fn=<MeanBackward0>)
Iter: 2690 tensor(6.4219, grad_fn=<MeanBackward0>) tensor(66.6068, grad_fn=<MeanBackward0>)
Iter: 2700 tensor(6.4227, grad_fn=<MeanBackward0>) tensor(52.8721, grad_fn=<MeanBackward0>)
Iter: 2710 tensor(6.4228, grad_fn=<MeanBackward0>) tensor(62.4463, grad_fn=<MeanBackward0>)
Iter: 2720 tensor(6.4229, grad_fn=<MeanBackward0>) tensor(49.4235, grad_fn=<MeanBackward0>)
Iter: 2730 tensor(6.4222, grad_fn=<MeanBackward0>) tensor(71.0169, grad_fn=<MeanBackward0>)
Iter: 2740 tensor(6.4222, grad_fn=<MeanBackward0>) tensor(60.7906, grad_fn=<MeanBackward0>)
Iter: 2750 tensor(6.4221, grad_fn=<MeanBackward0>) tensor(103.6647, grad_fn=<MeanBackward0>)
Iter: 2760 tensor(6.4216, grad_fn=<MeanBackward0>) tensor(90.1652, grad_fn=<MeanBackward0>)
Iter: 2770 tensor(6.4210, grad_fn=<MeanBackward0>) tensor(78.0834, grad_fn=<MeanBackward0>)
Iter: 2780 tensor(6.4213, grad_fn=<MeanBackward0>) tensor(98.0428, grad_fn=<Me

Iter: 3570 tensor(6.4771, grad_fn=<MeanBackward0>) tensor(102.0363, grad_fn=<MeanBackward0>)
Iter: 3580 tensor(6.4777, grad_fn=<MeanBackward0>) tensor(84.4899, grad_fn=<MeanBackward0>)
Iter: 3590 tensor(6.4780, grad_fn=<MeanBackward0>) tensor(132.9470, grad_fn=<MeanBackward0>)
Iter: 3600 tensor(6.4779, grad_fn=<MeanBackward0>) tensor(56.9877, grad_fn=<MeanBackward0>)
Iter: 3610 tensor(6.4776, grad_fn=<MeanBackward0>) tensor(67.3428, grad_fn=<MeanBackward0>)
Iter: 3620 tensor(6.4763, grad_fn=<MeanBackward0>) tensor(37.2614, grad_fn=<MeanBackward0>)
Iter: 3630 tensor(6.4751, grad_fn=<MeanBackward0>) tensor(40.1207, grad_fn=<MeanBackward0>)
Iter: 3640 tensor(6.4745, grad_fn=<MeanBackward0>) tensor(46.6972, grad_fn=<MeanBackward0>)
Iter: 3650 tensor(6.4728, grad_fn=<MeanBackward0>) tensor(42.8446, grad_fn=<MeanBackward0>)
Iter: 3660 tensor(6.4715, grad_fn=<MeanBackward0>) tensor(144.1044, grad_fn=<MeanBackward0>)
Iter: 3670 tensor(6.4704, grad_fn=<MeanBackward0>) tensor(54.5252, grad_fn=<M

Iter: 4460 tensor(6.5126, grad_fn=<MeanBackward0>) tensor(54.3563, grad_fn=<MeanBackward0>)
Iter: 4470 tensor(6.5128, grad_fn=<MeanBackward0>) tensor(130.6933, grad_fn=<MeanBackward0>)
Iter: 4480 tensor(6.5129, grad_fn=<MeanBackward0>) tensor(207.9514, grad_fn=<MeanBackward0>)
Iter: 4490 tensor(6.5132, grad_fn=<MeanBackward0>) tensor(49.6945, grad_fn=<MeanBackward0>)
Iter: 4500 tensor(6.5134, grad_fn=<MeanBackward0>) tensor(43.6752, grad_fn=<MeanBackward0>)
Iter: 4510 tensor(6.5144, grad_fn=<MeanBackward0>) tensor(87.1891, grad_fn=<MeanBackward0>)
Iter: 4520 tensor(6.5144, grad_fn=<MeanBackward0>) tensor(35.9060, grad_fn=<MeanBackward0>)
Iter: 4530 tensor(6.5147, grad_fn=<MeanBackward0>) tensor(76.7299, grad_fn=<MeanBackward0>)
Iter: 4540 tensor(6.5151, grad_fn=<MeanBackward0>) tensor(63.5159, grad_fn=<MeanBackward0>)
Iter: 4550 tensor(6.5146, grad_fn=<MeanBackward0>) tensor(120.3904, grad_fn=<MeanBackward0>)
Iter: 4560 tensor(6.5150, grad_fn=<MeanBackward0>) tensor(66.6169, grad_fn=<M

Iter: 5350 tensor(6.5452, grad_fn=<MeanBackward0>) tensor(97.0097, grad_fn=<MeanBackward0>)
Iter: 5360 tensor(6.5456, grad_fn=<MeanBackward0>) tensor(69.8398, grad_fn=<MeanBackward0>)
Iter: 5370 tensor(6.5448, grad_fn=<MeanBackward0>) tensor(49.3596, grad_fn=<MeanBackward0>)
Iter: 5380 tensor(6.5447, grad_fn=<MeanBackward0>) tensor(114.3328, grad_fn=<MeanBackward0>)
Iter: 5390 tensor(6.5452, grad_fn=<MeanBackward0>) tensor(83.6913, grad_fn=<MeanBackward0>)
Iter: 5400 tensor(6.5462, grad_fn=<MeanBackward0>) tensor(109.6748, grad_fn=<MeanBackward0>)
Iter: 5410 tensor(6.5466, grad_fn=<MeanBackward0>) tensor(53.3241, grad_fn=<MeanBackward0>)
Iter: 5420 tensor(6.5455, grad_fn=<MeanBackward0>) tensor(83.3574, grad_fn=<MeanBackward0>)
Iter: 5430 tensor(6.5454, grad_fn=<MeanBackward0>) tensor(89.5751, grad_fn=<MeanBackward0>)
Iter: 5440 tensor(6.5465, grad_fn=<MeanBackward0>) tensor(70.1875, grad_fn=<MeanBackward0>)
Iter: 5450 tensor(6.5481, grad_fn=<MeanBackward0>) tensor(81.8863, grad_fn=<Me

Iter: 6240 tensor(6.5741, grad_fn=<MeanBackward0>) tensor(70.6173, grad_fn=<MeanBackward0>)
Iter: 6250 tensor(6.5738, grad_fn=<MeanBackward0>) tensor(76.1862, grad_fn=<MeanBackward0>)
Iter: 6260 tensor(6.5727, grad_fn=<MeanBackward0>) tensor(128.6130, grad_fn=<MeanBackward0>)
Iter: 6270 tensor(6.5724, grad_fn=<MeanBackward0>) tensor(34.9444, grad_fn=<MeanBackward0>)
Iter: 6280 tensor(6.5719, grad_fn=<MeanBackward0>) tensor(142.2448, grad_fn=<MeanBackward0>)
Iter: 6290 tensor(6.5728, grad_fn=<MeanBackward0>) tensor(64.4686, grad_fn=<MeanBackward0>)
Iter: 6300 tensor(6.5730, grad_fn=<MeanBackward0>) tensor(137.8466, grad_fn=<MeanBackward0>)
Iter: 6310 tensor(6.5715, grad_fn=<MeanBackward0>) tensor(118.0042, grad_fn=<MeanBackward0>)
Iter: 6320 tensor(6.5719, grad_fn=<MeanBackward0>) tensor(54.0185, grad_fn=<MeanBackward0>)
Iter: 6330 tensor(6.5721, grad_fn=<MeanBackward0>) tensor(139.6594, grad_fn=<MeanBackward0>)
Iter: 6340 tensor(6.5728, grad_fn=<MeanBackward0>) tensor(48.1436, grad_fn=

Iter: 7130 tensor(6.5765, grad_fn=<MeanBackward0>) tensor(70.1203, grad_fn=<MeanBackward0>)
Iter: 7140 tensor(6.5762, grad_fn=<MeanBackward0>) tensor(96.7979, grad_fn=<MeanBackward0>)
Iter: 7150 tensor(6.5756, grad_fn=<MeanBackward0>) tensor(77.3768, grad_fn=<MeanBackward0>)
Iter: 7160 tensor(6.5748, grad_fn=<MeanBackward0>) tensor(51.5387, grad_fn=<MeanBackward0>)
Iter: 7170 tensor(6.5752, grad_fn=<MeanBackward0>) tensor(143.1960, grad_fn=<MeanBackward0>)
Iter: 7180 tensor(6.5761, grad_fn=<MeanBackward0>) tensor(83.5955, grad_fn=<MeanBackward0>)
Iter: 7190 tensor(6.5757, grad_fn=<MeanBackward0>) tensor(154.0092, grad_fn=<MeanBackward0>)
Iter: 7200 tensor(6.5758, grad_fn=<MeanBackward0>) tensor(104.7434, grad_fn=<MeanBackward0>)
Iter: 7210 tensor(6.5760, grad_fn=<MeanBackward0>) tensor(229.1584, grad_fn=<MeanBackward0>)
Iter: 7220 tensor(6.5762, grad_fn=<MeanBackward0>) tensor(114.0755, grad_fn=<MeanBackward0>)
Iter: 7230 tensor(6.5759, grad_fn=<MeanBackward0>) tensor(69.7330, grad_fn=

In [91]:
y_pred=np.array(y0_mean)
loss_var=np.array(loss_vec)
iters=np.arange(0,len(y_pred),1)+1

In [92]:
import pandas as pd

In [93]:
df=pd.DataFrame()
df['iter']=iters
df['y_pred']=y_pred
df['loss_var']=loss_var

In [95]:
df.to_csv('eg3_trainedMethod1_data//method1_1.csv')